In [1]:
import graphlab

In [2]:
people = graphlab.SFrame('people_wiki.gl')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1544418795.log
INFO:graphlab.cython.cy_server:GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1544418795.log


This non-commercial license of GraphLab Create for academic use is assigned to xaero@y7mail.com and will expire on October 26, 2019.


### Q1. Compare top words according to word counts to TF-IDF: 

In the notebook we covered in the module, we explored two document representations: word counts and TF-IDF. Now, take a particular famous person, 'Elton John'. What are the 3 words in his articles with highest word counts? What are the 3 words in his articles with highest TF-IDF? These results illustrate why TF-IDF is useful for finding important words.

In [6]:
elton = people[people['name'] == 'Elton John']

In [7]:
elton['word_count'] = graphlab.text_analytics.count_words(elton['text'])

In [16]:
elton_word_count_table = elton[['word_count']].stack('word_count', new_column_name=['word','count']).sort('count',ascending=False)

In [17]:
elton_word_count_table

word,count
the,27
in,18
and,15
of,13
a,10
has,9
john,7
he,7
on,6
award,5


### Top 3 wordcount
the,
in,
and

In [19]:
people['word_count'] = graphlab.text_analytics.count_words(people['text'])

In [20]:
tfidf = graphlab.text_analytics.tf_idf(people['word_count'])

In [25]:
people['tfidf'] = tfidf

In [28]:
elton = people[people['name'] == 'Elton John']

In [29]:
elton[['tfidf']].stack('tfidf',new_column_name=['word','tfidf']).sort('tfidf',ascending=False)

word,tfidf
furnish,18.38947184
elton,17.48232027
billboard,17.3036809575
john,13.9393127924
songwriters,11.250406447
tonightcandle,10.9864953892
overallelton,10.9864953892
19702000,10.2933482087
fivedecade,10.2933482087
aids,10.262846934


### Top 3 words tf-idf
furnish, elton, bilboard

### Q2. Measuring distance
Elton John is a famous singer; let’s compute the distance between his article and those of two other famous singers. In this assignment, you will use the cosine distance, which one measure of similarity between vectors, similar to the one discussed in the lectures. You can compute this distance using the graphlab.distances.cosine function. What’s the cosine distance between the articles on ‘Elton John’ and ‘Victoria Beckham’? What’s the cosine distance between the articles on ‘Elton John’ and Paul McCartney’? Which one of the two is closest to Elton John? Does this result make sense to you?

In [31]:
victoria = people[people['name'] == 'Victoria Beckham']

In [32]:
paul = people[people['name'] == 'Paul McCartney']

In [33]:
graphlab.distances.cosine(elton['tfidf'][0],victoria['tfidf'][0])

0.9567006376655429

In [34]:
graphlab.distances.cosine(elton['tfidf'][0],paul['tfidf'][0])

0.8250310029221779

### Lower is closer so Obama is closer to Clinton.
Paul McCartney is closer to Elton John than is Victoria Beckham. See StackOverflow link here: 
https://stackoverflow.com/questions/1746501/can-someone-give-an-example-of-cosine-similarity-in-a-very-simple-graphical-wa

### Q3. Building nearest neighbors models with different input features and setting the distance metric
In the sample notebook, we built a nearest neighbors model for retrieving articles using TF-IDF as features and using the default setting in the construction of the nearest neighbors model

In [46]:
knn_model_tfidf = graphlab.nearest_neighbors.create(people, features=['tfidf'],label='name',distance='cosine')

Starting brute force nearest neighbors model training.

In [47]:
knn_model_word_count = graphlab.nearest_neighbors.create(people, features=['word_count'],label='name'distance='cosine',)

SyntaxError: invalid syntax (<ipython-input-47-6a29bf1b241d>, line 1)

In [48]:
knn_model_tfidf.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 114.178ms    |

| 0            | 34817   | 58.9409     | 1.12s        |

| Done         |         | 100         | 1.64s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,-2.22044604925e-16,1
0,Rod Stewart,0.717219667893,2
0,George Michael,0.747600998969,3
0,Sting (musician),0.747671954431,4
0,Phil Collins,0.75119324879,5


In [49]:
knn_model_word_count.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 15.091ms     |

| Done         |         | 100         | 868.199ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,0.0,1
0,Phil Collins,0.76399026764,2
0,Rod Stewart,0.773333333333,3
0,Annie Lennox,0.776623376623,4
0,Barry Gibb,0.780952380952,5


In [50]:
knn_model_tfidf.query(victoria)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 19.464ms     |

| 0            | 46081   | 78.0095     | 1.02s        |

| Done         |         | 100         | 1.30s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,1.11022302463e-16,1
0,David Beckham,0.548169610263,2
0,Stephen Dow Beckham,0.784986706828,3
0,Mel B,0.809585523409,4
0,Caroline Rush,0.819826422919,5


In [51]:
knn_model_word_count.query(victoria)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 13.234ms     |

| Done         |         | 100         | 841.075ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,0.0,1
0,Cheryl Cole,0.800586510264,2
0,Heidi Klum,0.810344827586,3
0,Simon Fuller,0.822742474916,4
0,Adele,0.824915824916,5
